In [1]:
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
from keras.applications import VGG16


Using TensorFlow backend.


In [3]:
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))


In [4]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [6]:
for layer in model.layers:
    layer.trainable = False

In [7]:
top_model = model.output
top_model = Flatten(name = "flatten")(top_model)
top_model = Dense(512, activation = "relu")(top_model)
top_model = Dense(256, activation = "relu")(top_model)
top_model = Dropout(0.3)(top_model)
top_model = Dense(2 , activation = "softmax")(top_model)


In [8]:
newmodel = Model(inputs=model.input , outputs=top_model)


In [9]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

In [12]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train_batchsize = 30
val_batchsize = 16

In [14]:
train_generator = train_datagen.flow_from_directory(
        'C:/Users/MAC/Desktop/MLOPS/DATASETS/datasets/160647_367971_bundle_archive_3/data/train_set',
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 

Found 2584 images belonging to 2 classes.


In [15]:
validation_generator = validation_datagen.flow_from_directory(
        'C:/Users/MAC/Desktop/MLOPS/DATASETS/datasets/160647_367971_bundle_archive_3/data/test_set',
        target_size=(224, 224 ),
        batch_size=val_batchsize,
        class_mode='categorical')

Found 725 images belonging to 2 classes.


In [16]:
from keras.optimizers import RMSprop


In [17]:
newmodel.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop( lr = 0.001 ),
              metrics = ['accuracy'])

In [18]:
nb_train_samples = 1097
nb_validation_samples = 272
batch_size = 10
history = newmodel.fit_generator(
    train_generator,
    steps_per_epoch = 50,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    epochs = 30)

Epoch 1/30
50/50 [==============================] - 30s 604ms/step - loss: 2.6293 - accuracy: 0.5421 - val_loss: 0.7568 - val_accuracy: 0.5579
Epoch 2/30
50/50 [==============================] - 27s 534ms/step - loss: 0.7476 - accuracy: 0.6060 - val_loss: 0.4150 - val_accuracy: 0.7387
Epoch 3/30
50/50 [==============================] - 24s 472ms/step - loss: 0.6693 - accuracy: 0.6201 - val_loss: 0.6199 - val_accuracy: 0.7778
Epoch 4/30
50/50 [==============================] - 27s 535ms/step - loss: 0.6056 - accuracy: 0.6833 - val_loss: 0.6025 - val_accuracy: 0.7482
Epoch 5/30
50/50 [==============================] - 24s 490ms/step - loss: 0.5985 - accuracy: 0.7062 - val_loss: 0.3745 - val_accuracy: 0.8403
Epoch 6/30
50/50 [==============================] - 31s 627ms/step - loss: 0.5733 - accuracy: 0.7107 - val_loss: 0.3275 - val_accuracy: 0.8195
Epoch 7/30
50/50 [==============================] - 26s 516ms/step - loss: 0.5286 - accuracy: 0.7368 - val_loss: 0.1615 - val_accuracy: 0.7838

In [22]:
result_accuracy = history.history['accuracy']

In [23]:
newmodel.save('Facial Recognition_mac.h5')

In [24]:
newmodel.save('Facial Recognition_mac.xml')